In [3]:
import sympy
from openLoop.utilities.ipynb.displays import *
from openLoop.utilities.ipynb.ipy_sympy import *
import scipy.linalg


import numpy.testing as np_test
import declarative

from test_SVD import SVD_gen_check, gen_rand_unitary
from openLoop.system import DAG_algorithm
from openLoop.system import SRE_matrix_algorithms
from openLoop.system import scisparse_algorithm

import timeit

asavefig.org_subfolder = 'plots'

In [20]:
from functools import reduce
def SVD_compare_error(
    N = 10,
    length = 10,
    solver =  DAG_algorithm,
    N_in = None,
    N_out = None,
):
    U = gen_rand_unitary(N = N, length = length)
    V = gen_rand_unitary(N = N, length = length)

    seq = dict()
    req = dict()
    edge_map = dict()
    S_diags = []
    for idx in range(N):
        s_diag = 10**(-5 + 10 * np.random.random(length))
        edge_map[idx, idx] = s_diag
        S_diags.append(s_diag)
        seq[idx] = set([idx])
        req[idx] = set([idx])
    S = seq, req, edge_map
    condition = reduce(np.maximum, S_diags) / reduce(np.minimum, S_diags)

    M = SRE_matrix_algorithms.matrix_mult_sre(
        SRE_matrix_algorithms.matrix_mult_sre(U, S), V
    )

    SRE_matrix_algorithms.check_sre(M)
    sparsity = SRE_matrix_algorithms.SRE_count_sparsity(M)
    print("SPARSITY FRAC: ", sparsity)

    if N_in is None:
        inputs_set = set(range(N))
    else:
        inputs_set = set(range(N-N_in, N))
    if N_out is None:
        outputs_set = set(range(N))
    else:
        mid = N_out // 2
        outputs_set = set(range(0, mid)) | set(range(N-(N_out - mid), N))

    Mseq, Mreq, Medge_map = SRE_matrix_algorithms.copy_sre(M)
    print(solver)
    time_start = timeit.default_timer()
    sbunch = solver.inverse_solve_inplace(
        seq = Mseq,
        req = Mreq,
        edge_map = Medge_map,
        inputs_set = inputs_set,
        outputs_set = outputs_set,
        verbose = True,
        negative = False,
    )
    time_end = timeit.default_timer()
    b = declarative.Bunch( 
        time = time_end - time_start,
        length = length,
    )
    b.update(sparsity)
    return b

In [21]:
mats = []
for N in [10, 30, 100, 300]:
    for length in [10, 100, 1000, 10000]:
        for inst in range(3):
            r = SVD_compare_error(
                N = N,
                length = length,
            )
            mats.append(r)

SPARSITY FRAC:  {'density_lin': 1.9, 'density_sq': 0.19, 'Nnodes': 10, 'Nedges': 19}
<module 'openLoop.system.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py'>
TRIVIAL STAGE, REMAINING 10
TRIVIAL STAGE, REMAINING 7
BADGUY STAGE, REMAINING 7
SPARSITY FRAC:  {'density_lin': 2.8, 'density_sq': 0.28, 'Nnodes': 10, 'Nedges': 28}
<module 'openLoop.system.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py'>
TRIVIAL STAGE, REMAINING 10
TRIVIAL STAGE, REMAINING 7
BADGUY STAGE, REMAINING 7
SPARSITY FRAC:  {'density_lin': 3.3, 'density_sq': 0.33, 'Nnodes': 10, 'Nedges': 33}
<module 'openLoop.system.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/openLoop/openLoop/system/DAG_algorithm.py'>
TRIVIAL STAGE, REMAINING 10
TRIVIAL STAGE, REMAINING 10
BADGUY STAGE, REMAINING 10
SPARSITY FRAC:  {'density_lin': 2.8, 'density_sq': 0.28, 'Nnodes': 10, 'Nedges': 28}
<module 'openLoop.system.

In [26]:
axB = mplfigB(Nrows=1)
color_by_len = {
    10 : 'blue',
    100 : 'green',
    1000 : 'purple',
    10000 : 'red',
}
for r in mats:
    axB.ax0.scatter(
        r.Nnodes, 
        r.time,
        color = color_by_len[r.length],
    )
axB.ax0.set_xscale('log')
axB.ax0.set_yscale('log')
axB.ax0.set_xlim(9, 400)
axB.save('timing_by_nodes_dir')

figure: plots/timing_by_nodes_dir.png
[[file:plots/timing_by_nodes_dir.png]]


In [27]:
axB = mplfigB(Nrows=1)
for r in mats:
    axB.ax0.scatter(
        r.Nnodes, 
        r.time / r.Nnodes**2,
        color = color_by_len[r.length],
    )
axB.ax0.set_xscale('log')
axB.ax0.set_yscale('log')
axB.ax0.set_xlim(9, 400)
axB.save('timing_by_nodes_Nsq')

figure: plots/timing_by_nodes_Nsq.png
[[file:plots/timing_by_nodes_Nsq.png]]


In [28]:
axB = mplfigB(Nrows=1)
for r in mats:
    axB.ax0.scatter(
        r.Nnodes, 
        r.time / r.length,
        color = color_by_len[r.length],
    )
axB.ax0.set_xscale('log')
axB.ax0.set_yscale('log')
axB.ax0.set_xlim(9, 400)
axB.save('timing_by_nodes_rlen')

figure: plots/timing_by_nodes_rlen.png
[[file:plots/timing_by_nodes_rlen.png]]
